In [3]:
import pandas as pd
from fuzzywuzzy import fuzz, process

In [4]:
SPAC_Alpha_data = pd.read_excel("C:\\Users\\msnin\\OneDrive\\Desktop\\SPAC Research\\SPACAlphadata.xlsx", sheet_name='testing')
data_overlap = pd.read_excel("C:\\Users\\msnin\\OneDrive\\Desktop\\SPAC Research\\SPACAlphadata.xlsx", sheet_name='Data_Overlap')

In [5]:
def exact_match(row):
    ticker_ss = str(row['Ticker - SS']).strip()

    spac_ticker_choices = SPAC_Alpha_data['SPAC Ticker'].dropna().astype(str).tolist()
    newco_ticker_choices = SPAC_Alpha_data['Newco Ticker'].dropna().astype(str).tolist()
    newcow_ticker_choices = SPAC_Alpha_data['Newco Warrant Ticker'].dropna().astype(str).tolist()

    spac_ticker_match = process.extractOne(ticker_ss, spac_ticker_choices, scorer=fuzz.ratio)
    newco_ticker_match = process.extractOne(ticker_ss, newco_ticker_choices, scorer=fuzz.ratio)
    newcow_ticker_match = process.extractOne(ticker_ss, newcow_ticker_choices, scorer=fuzz.ratio)

    if spac_ticker_match[1] == 100:
        return spac_ticker_match[0]
    elif newco_ticker_match[1] == 100:
        return newco_ticker_match[0]
    elif newcow_ticker_match[1] == 100:
        return newcow_ticker_match[0]
    else:
        return None

data_overlap['MatchwSTicker'] = data_overlap.apply(exact_match, axis=1)


In [6]:
def fuzzy_match(row):
    #if the exact matching didn't work then run a fuzzy match
    if pd.isnull(row['MatchwSTicker']):
        ticker = row['Ticker - SS']
        spac_tickers = SPAC_Alpha_data['SPAC Ticker'].tolist()

        for spac_ticker in spac_tickers:
            if fuzz.token_sort_ratio(ticker, spac_ticker) >= 80: #threshold 
                return spac_ticker
    return None

data_overlap['MatchwNTicker'] = data_overlap.apply(fuzzy_match, axis=1)


In [7]:
display(data_overlap[['Ticker - SS', 'MatchwSTicker', 'MatchwNTicker']])
data_overlap.to_excel("data_overlap_matched", index=False)

,Ticker - SS,MatchwSTicker,MatchwNTicker
0,CPTN,CPTN,None
1,DMTK,DMTK,None
2,TIBR,TIBR,None
3,USWS,USWS,None
4,LTRY,LTRY,None
...,...,...,...
1046,PLTN,None,LTN
1047,PTWOU,None,None
1048,MOBV,None,None
1049,ATMVU,None,None
